In [1]:
# The script should operate from the repository's parent directory.
import os
print(os.getcwd())
os.chdir('..')
print(os.getcwd())

/Users/Tom/Documents/projects/pinging/spiking_visnet/scripts
/Users/Tom/Documents/projects/pinging/spiking_visnet


In [ ]:
import spiking_visnet.plot as pl
from spiking_visnet.utils.sparsify import save_as_sparse
from spiking_visnet.analysis import utils as an_utils
from spiking_visnet.analysis import input_encoding as an_inp
from spiking_visnet.analysis import activity as an_activity
from spiking_visnet import save

In [ ]:
output_dir = 'output/default'

# Run:

```
python -m spiking_visnet scripts/tests_stdp/params/default.yml -i scripts/tests_stdp/input -o scripts/tests_stdp/output
```

# Plot results

### Activity movies

In [ ]:
#Plot stim
session_name = 'learning'

# Load first filter of full_stim
stim = save.load_session_stim(output_dir, session_name)
stim = stim[:,0,:,:]
p = pl.init()
pl.animate(p, stim, fps=100)

In [ ]:
#plot input layer
layer = 'input_layer'
pop = 'parrot_neuron'

arr = save.load_activity(output_dir, layer, pop)
p = pl.init()
pl.animate(p, arr, fps=100)

In [ ]:
#plot exc layer
layer = 'l1'
pop = 'l1_exc'
variable = 'spikes'
arr = save.load_activity(output_dir, layer, pop)
p = pl.init()
pl.animate(p, arr, fps=100)

In [ ]:
#plot inh layer
layer = 'l1'
pop = 'l1_inh'
variable = 'spikes'
arr = save.load_activity(output_dir, layer, pop)
p = pl.init()
pl.animate(p, arr, fps=100)

### Plot mean activity

In [ ]:
# Define which variable we plot and analyse
variable = 'spikes'
# Define which session we plot and analyse.
# The data considered is the data from the start of the earliest session to the end of the latest
plot_sessions = ['warmup', 'learning']
rate_sessions = ['learning']
cv_sessions = ['learning']

In [ ]:
# Get times
session_times = save.load_session_times(output_dir)
def union_session_times(sessions):
    """Return range between earliest and latest time of the sessions in the list"""
    session_range_list = [session_times[session] for session in sessions]
    return range(min([min(times) for times in session_range_list]),
                 max([max(times) for times in session_range_list]) + 1)

plot_period = union_session_times(plot_sessions)
rate_period = union_session_times(rate_sessions)
cv_period = union_session_times(cv_sessions)

In [ ]:
# Input layer

inp = an_utils.load_activity('input_layer', 'parrot_neuron', output_dir)

print('Activity raster-plot:')
pl.show_activity_raster(inp[plot_period], plot_cols=[0,1])

print('Firing rate:')
rates = an_activity.firing_rate(inp[rate_period])
pl.show_mean_min_max(rates)

print('CV distribution:')
pl.show_distribution(an_activity.all_cv(inp))

In [ ]:
# l1_exc

l1_exc = an_utils.load_activity('l1', 'l1_exc', output_dir)

print('Activity raster-plot:')
pl.show_activity_raster(l1_exc[plot_period])

print('Firing rate:')
rates = an_activity.firing_rate(l1_exc[rate_period])
pl.show_mean_min_max(rates)

print('CV distribution:')
pl.show_distribution(an_activity.all_cv(l1_exc))

In [ ]:
l1_inh = an_utils.load_activity('l1', 'l1_inh', output_dir, variable=plot_variable)

print('Activity raster-plot:')
pl.show_activity_raster(l1_inh[plot_period])

print('Firing rate:')
rates = an_activity.firing_rate(l1_inh[rate_period])
pl.show_mean_min_max(rates)

print('CV distribution:')
# pl.show_distribution(np.ones((2,2)))
pl.show_distribution(an_activity.all_cv(l1_inh[cv_period]))

### Plot synapses 

In [ ]:
#Load input connections
synapses_filename = 'synapses_from_input_layer_parrot_neuron_to_l1_l1_exc_weight.npy'
synapses = np.load(join(output_dir, synapses_filename))

In [ ]:
print('All input synapses, by units of target layer:')
pl.show_array_of_images(synapses, plot_rows=range(5), plot_cols=range(5))

In [ ]:
print('All synapses to all units: ')
pl.show_distribution(synapses, min_value=0)

## Train/test accuracy: predict using mean activity of a pool of neurons

In [ ]:
pretest_session_name = 'warmup'
test_session_name = 'learning'
activity_filename = 'l1_l1_exc_spikes.npy'
time_per_movie = 10

# Load labels
labels_pretest = save.load_labels(output_dir, pretest_session_name)
labels_test = save.load_labels(output_dir, test_session_name)

# Load full activity
activity = np.load(join(output_dir, activity_filename))
# Load session times to get session activity
session_times = save.load_session_times(output_dir)
activity_pretest = activity[session_times[pretest_session_name]]
activity_test = activity[session_times[test_session_name]]

In [ ]:
# Get the preferred label of each unit
best_label = an.get_best_label(activity_pretest, labels_pretest)

# How many units prefer each label?
unique_bests, count = np.unique(best_label, return_counts=True)
print('is this balanced? ', count)

In [ ]:
acc = an.test_network_predictions(activity_test, labels_test, best_label, time_per_movie)
print(acc)